In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [2]:
in_dir = "./data/in"

In [3]:
os.listdir(in_dir)

['DOGEUSDT.csv',
 '.DS_Store',
 'DASHUSDT.csv',
 'ZRXUSDT.csv',
 'SOLUSDT.csv',
 'LTCUSDT.csv',
 'ETHUSDT.csv',
 'BTCUSDT.csv',
 'ADAUSDT.csv',
 'XRPUSDT.csv',
 'XLMUSDT.csv']

In [4]:
candle_paths = []

for filename in os.listdir(in_dir):
    extension = os.path.splitext(filename)[1]
    
    if extension == ".csv":
        candle_paths.append(os.path.join(in_dir, filename))

In [5]:
asset_col_name = "Měna"
candle_count_col_name = "Počet svíček"
from_col_name = "Od (den)"
to_col_name = "Do (den)"
duration_col_name = "Doba (roky)"
candle_overview = pd.DataFrame(columns = [asset_col_name, candle_count_col_name, from_col_name,
                                          to_col_name, duration_col_name])

In [6]:
def read_candles(path):
    column_names = ["time", "open", "high", "low", "close", "volume"]
    candles = pd.read_csv(path, sep = '|', names=column_names, usecols=range(6), header=None)
    candles.set_index("time", inplace=True)
    candles.index = pd.to_datetime(candles.index,unit='s')
    return candles

In [7]:
def itos(num):
    out = '{:,}'.format(num)
    return out.replace(",", " ")

In [8]:
def ftos(num):
    out = "{:,.1f}".format(num).replace(',', ' ')
    return out.replace(".", ",")

In [9]:
def datetime_to_string(time):
    return time.strftime('%d. %m. %Y')

In [10]:
quote_name = "USDT"

for candle_path in candle_paths:
    candles = read_candles(candle_path)
    candle_count = itos(len(candles))
    candle_from = candles.index[0]
    candle_to = candles.index[-1]
    duration = (candle_to-candle_from).total_seconds()/timedelta(days=365).total_seconds()
    
    filename = os.path.basename(candle_path)
    name = os.path.splitext(filename)[0]
    asset = name.replace(quote_name, "")
    
    candle_overview.loc[len(candle_overview)] = {
        asset_col_name : asset,
        candle_count_col_name : candle_count,
        from_col_name : datetime_to_string(candle_from),
        to_col_name : datetime_to_string(candle_to),
        duration_col_name : ftos(duration)
    }

In [11]:
candle_overview

,Měna,Počet svíček,Od (den),Do (den),Doba (roky)
0,DOGE,1 979 751,05. 07. 2019,12. 04. 2023,"3,8"
1,DASH,2 122 039,28. 03. 2019,12. 04. 2023,"4,0"
2,ZRX,2 160 979,28. 02. 2019,11. 04. 2023,"4,1"
3,SOL,1 400 292,11. 08. 2020,11. 04. 2023,"2,7"
4,LTC,2 794 419,13. 12. 2017,12. 04. 2023,"5,3"
5,ETH,2 963 365,17. 08. 2017,12. 04. 2023,"5,7"
6,BTC,2 963 250,17. 08. 2017,11. 04. 2023,"5,7"
7,ADA,2 616 803,17. 04. 2018,12. 04. 2023,"5,0"
8,XRP,2 591 038,04. 05. 2018,11. 04. 2023,"4,9"
9,XLM,2 552 037,31. 05. 2018,11. 04. 2023,"4,9"


In [12]:
# sort by asset name
candle_overview = candle_overview.sort_values(asset_col_name)

In [13]:
report_dir = "./data/report"
candle_overview.to_csv(os.path.join(report_dir, "candles-overview.csv"), index=False)